In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjXDhUcwibguXnJO_7-aXZSnPnpBzpQwO0awdHM_Yw9ZaLT1qKGD-A
Mounted at /content/drive


In [3]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip uninstall openpyxl
!pip install openpyxl

     |████████████████████████████████| 132 kB 14.4 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219329 sha256=7816c832240e69c874b676008d3c4f779982a04b35a6092ebba5a0ef9fd0a9d8
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5
     |████████████████████████████████| 11.3 MB 15.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


Found existing installation: openpyxl 2.5.9
Uninstalling openpyxl-2.5.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/openpyxl-2.5.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/openpyxl/*
Proceed (y/n)? y
  Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 242 kB 14.6 MB/s 


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pickle5 as pickle
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import joblib



# 시계열로 예측한 예측중량 독립변수로 사용
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/중장기수주예측.pkl', "rb") as fh:
  중장기수주예측 = pickle.load(fh)
# std_date = datetime.today().strftime('%Y-%m-%d')
# std_date = pd.to_datetime(today,format='%Y-%m-%d') ##실제 사용할 때는 today를 std_date로 
std_date = datetime.strptime("2021-04-25", "%Y-%m-%d")
end_date = std_date + relativedelta(days=95)
중장기수주예측 = 중장기수주예측[(중장기수주예측['납기일자']>=std_date) & (중장기수주예측['납기일자']<=end_date)]

df = 중장기수주예측.copy()
df.set_index('납기일자',inplace=True)
df['예측중량'] = np.log1p(df['예측중량'])
df.fillna(0,inplace=True)


# std_date 기준으로 최근 2년치 건축착공면적의 월별 평균을 독립변수로 사용
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/건축착공면적.pkl', "rb") as fh:
  건축착공면적 = pickle.load(fh)
건축착공면적['TIME'] = pd.to_datetime(건축착공면적['TIME'],format='%Y%m')
# std_date = datetime.today().strftime('%Y-%m-%d')
# std_date = pd.to_datetime(today,format='%Y-%m-%d')  ##실제 사용할 때는 today를 std_date로 
std_date = datetime.strptime('2021-04-25', '%Y-%m-%d')
start_date = std_date - relativedelta(years=2)

def cat(df):
  df = df[df['TIME']>=start_date]
  df['month'] = df.TIME.dt.month
  df = pd.DataFrame(df.groupby('month')['건축착공면적'].mean())
  df.reset_index(inplace=True)
  return df

건축착공면적 = cat(건축착공면적)

df.reset_index(inplace=True)
df['month'] = df['납기일자'].dt.month
# data['year'] = data['납기일자'].dt.year
df = pd.merge(df, 건축착공면적, on='month', how='left')
df.set_index('납기일자',inplace=True)

# 원핫인코딩
df = pd.get_dummies(df)
df.reset_index(inplace=True)
X_test = df.drop(['납기일자','month'], axis =1)


# 저장한 모델 불러오기
grid_rf = joblib.load('/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save_final/grid_rf.joblib.dat')

# 예측
pred_rf = grid_rf.predict(X_test)


# 예측량과 납기일자, 제품명 concat
pred = pd.DataFrame(pred_rf,columns=['생산량'])
pred['생산량'] = np.ceil(np.expm1(pred.생산량))
중장기수주예측 = 중장기수주예측[['납기일자','제품명']]
중장기수주예측.reset_index(drop=True,inplace=True)
생판계획 = pd.concat([중장기수주예측,pred],axis=1)

생판계획.sort_values(by='납기일자',ascending=True,inplace=True)
생판계획.reset_index(drop=True,inplace=True)
생판계획

,납기일자,제품명,생산량
0,2021-04-25,AE,1099.0
1,2021-04-25,SRE-200,12749.0
2,2021-04-25,SRE-110,2747.0
3,2021-04-25,PR1000,12367.0
4,2021-04-25,PEMA-SR5000F,4521.0
...,...,...,...
1680,2021-07-29,CSA4000,5718.0
1681,2021-07-29,AE,835.0
1682,2021-07-29,PEMA-SR2000A,5040.0
1683,2021-07-29,PEMA-HR1500,19853.0
